In [1]:
with open("requirements.txt", "w") as f:
    f.write("kfp==1.4.0\n")
    
!pip install -r requirements.txt  --upgrade --user

In [3]:
import kfp
import kfp.dsl as dsl


@dsl.pipeline(
    name="VolumeOp Sequential",
    description="Show how to use volumnOp"
)
def volumeop_sequential():
    vop = dsl.VolumeOp(
        name="mypvc",
        resource_name="newpvc",
        size="10Gi",
        modes=dsl.VOLUME_MODE_RWM
    )

    step1 = dsl.ContainerOp(
        name="step1",
        image="library/bash:4.4.23",
        command=["sh", "-c"],
        arguments=["echo 1|tee /data/file1"],
        pvolumes={"/data": vop.volume}
    )

    step2 = dsl.ContainerOp(
        name="step2",
        image="library/bash:4.4.23",
        command=["sh", "-c"],
        arguments=["cp /data/file1 /data/file2"],
        pvolumes={"/data": step1.pvolume}
    )

    step3 = dsl.ContainerOp(
        name="step3",
        image="library/bash:4.4.23",
        command=["cat", "/mnt/file1", "/mnt/file2"],
        pvolumes={"/mnt": step2.pvolume}
    )



In [4]:
kfp.compiler.Compiler().compile(volumeop_sequential, "helloworld.zip")

/home/jovyan/.local/lib/python3.8/site-packages/kfp/dsl/_container_op.py:1032: FutureWarning: Please create reusable components instead of constructing ContainerOp instances directly. Reusable components are shareable, portable and have compatibility and support guarantees. Please see the documentation: https://www.kubeflow.org/docs/pipelines/sdk/component-development/#writing-your-component-definition-file The components can be created manually (or, in case of python, using kfp.components.create_component_from_func or func_to_container_op) and then loaded using kfp.components.load_component_from_file, load_component_from_uri or load_component_from_text: https://kubeflow-pipelines.readthedocs.io/en/stable/source/kfp.components.html#kfp.components.load_component_from_file
  warnings.warn(
